In [114]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd

In [115]:
#reading rheast and extracting districts
rheast= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
pre=pd.read_csv('data/preharvest/preharvest_cleaned.csv')
post=pd.read_csv('data/postharvest/postharvest_cleaned.csv')
# post['province']=post['province'].replace(['northwest'], 'northwest')

In [116]:
# # Read shapefiles and rename columns
# # country_shapefile = gpd.read_file("data/shp/gadm36_ZMB_0.shp")
# # district_shp = gpd.read_file("data/shp/gadm36_ZMB_2.shp")


# # Rename kenya_shp 
# district_shp=district_shp.rename(columns={'NAME1':'cname'})
# # district_shp['cname']=district_shp['NAME_2']
# # len(district_shp.cname.unique())
# district_shp['cname']=district_shp['cname'].str.lower()
# district_shp['cname']=district_shp['cname'].str.replace('-', '', regex=True)
# district_shp['cname']=district_shp['cname'].str.strip()
# len(district_shp.cname.unique())

In [117]:
# # remove all white spaces, turn to lower case and
rheast.cname=rheast.cname.str.lower()
rheast.cname = rheast.cname.str.lstrip()
rheast.cname=rheast.cname.str.replace('-', '', regex=True)
rheast.cname=rheast.cname.str.replace(' ', '', regex=True)
# post

In [118]:
rheast['district']=rheast['cname']

In [119]:
#Extracting Districts
pre_list_districts= pre['district'].to_list()
pre_set_districts=set(pre_list_districts)
pre_districts=list(pre_set_districts)
rheast_list_districts=rheast['district'].to_list()
rheast_set_districts=set(rheast_list_districts)
rheas_districts=list(rheast_set_districts)

In [136]:
#Finding common districts in rheas and preharvest rheast
districts=[]
for i in pre_districts:
    if i in rheas_districts:
        districts.append(i)
    else:
        print(i)

missing_districts=[]
for j in pre_districts:
    if j not in  rheas_districts:
        missing_districts.append(j)

chienge
milenge
shang'ombo
kapiri-mposhi
itezhi-tezhi


In [123]:
#changing date columns
rheast['planting']=pd.to_datetime(rheast['planting'], format='ISO8601')
rheast['harvest'] = pd.to_datetime(rheast['harvest'], format='ISO8601')
rheast['year'] = rheast['planting'].dt.year
rheast['yield']=rheast['gwad']/1000


In [144]:
#Derive seasons from the data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
rheast['season']= rheast['planting'].dt.month.map(month_to_season)

# Extract data only for the planting season
rheast_model= rheast.loc[rheast['season']==1]
rheast_data=rheast_model.groupby(['district','year', 'season'], as_index=False).agg({'yield': 'mean'})

pre1=pre.groupby(['district', 'year'], as_index=False).agg({'yield': 'mean'})
# rheast_data[['district', 'year', 'yield']].to_csv('rheas_dssat_model.csv')
rheast_data2=rheast_data

In [145]:

distric_metrics1=[]
for i in districts:
    pre2=pre1.loc[pre1['district']==i].reset_index(drop=True)
    rheast_data3=rheast_data2.loc[rheast_data2['district']==i].reset_index(drop=True)
    #  pearson's correlation
    corr=rheast_data3['yield'].corr(pre2['yield'])
    # # #root mean square error
    rmse = np.sqrt(((rheast_data3['yield']-pre2['yield'])**2).mean())
    # # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_data3['yield']-rheast_data3['yield'].mean())
                       -(pre2['yield']-pre2['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_data3['yield']-pre2['yield']).mean()
    distric_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
# convert into dataframe and rename columns
district_statistics1=pd.DataFrame(distric_metrics1)
district_statistics1=district_statistics1.rename({0:'district',1: 'correlation',
                                                  2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
district_statistics1

,district,correlation,rmse,ubrmse,bias
0,gwembe,0.339743,2.013938,0.763381,1.864348
1,ikelenge,0.106779,1.216597,0.360101,1.163100
2,siavonga,0.687631,2.309958,0.360412,2.282264
3,mporokoso,0.213101,1.052670,0.303068,1.009378
4,mungwi,0.485762,0.710784,0.198287,0.684018
...,...,...,...,...,...
65,kabwe,0.503057,1.109900,0.664287,0.891685
66,lundazi,-0.327565,1.565040,0.436394,1.504113
67,chipata,0.234684,1.717924,0.236006,1.702008
68,kabompo,-0.364999,1.286977,0.413805,1.219390


In [126]:
# Province Level
#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('../zambia/data/shp/zambia_districts.shp')
zambia_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_1.shp')
zambia_district_shp .rename(columns={'DISTRICT':'district', 'PROVINCE':'province'}, inplace=True)
# merge the yields and shapefiles by provinces
pre_shp= pd.merge(zambia_district_shp, pre1, on="district")
post_shp= pd.merge(zambia_district_shp, post, on="province")

rheast_shp= pd.merge(zambia_district_shp, rheast_data2, on="district")
# rheasc_shp= pd.merge(zambia_district_shp, rheast_data2 on="p")
pre_province=pre_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
post_province=post_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province=rheast_shp.groupby(['province', 'year'], as_index=False).agg({'yield': 'mean'})
rheast_province

#Extracting Provinces
pre_list_provinces= pre_province['province'].to_list()
pre_set_provinces=set(pre_list_provinces)
pre_provinces=list(pre_set_provinces)
rheast_list_provinces=rheast_province['province'].to_list()
rheast_set_provinces=set(rheast_list_provinces)
rheast_provinces=list(rheast_set_provinces)

#list of provinces
provinces=[]
for i in pre_provinces:
    if i in rheast_provinces:
        provinces.append(i)

In [143]:
provinces=['central', 'copperbelt', 'eastern', 'luapula', 'lusaka',
       'muchinga', 'northern', 'northwestern', 'southern', 'western']

province_metrics1=[]
for i in provinces:
    pre_province1=pre_province.loc[pre_province['province']==i].reset_index(drop=True)
    rheast_province1=rheast_province.loc[rheast_province['province']==i].reset_index(drop=True)  
    #  pearson's correlation
    corr=rheast_province['yield'].corr(pre_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province1['yield']-pre_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province1['yield']-rheast_province1['yield'].mean())
                       -(pre_province1['yield']-pre_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province1['yield']-pre_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics1=pd.DataFrame(province_metrics1)
province_statistics1=province_statistics1.rename({0:'province',1: 'correlation',2: 
                                                  'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics1.to_csv('preharvest_dssat_province_level_metrics.csv')
province_statistics1#.to_csv('zambia_preharvest_rheas__province_level_metrics.csv')

,province,correlation,rmse,ubrmse,bias
0,central,NaN,NaN,NaN,NaN
1,copperbelt,NaN,NaN,NaN,NaN
2,eastern,NaN,NaN,NaN,NaN
3,luapula,NaN,NaN,NaN,NaN
4,lusaka,NaN,NaN,NaN,NaN
5,muchinga,NaN,NaN,NaN,NaN
6,northern,NaN,NaN,NaN,NaN
7,northwestern,NaN,NaN,NaN,NaN
8,southern,NaN,NaN,NaN,NaN
9,western,NaN,NaN,NaN,NaN


In [128]:
# Postharvest and rheas common years 
post_list_years= post['year'].to_list()
years_set=set(post_list_years)
post_harvest_years=list(years_set)
post_harvest_years

# df1=rheast_province.loc[rheast_province['year']==2016]
rheast_province2=pd.DataFrame(columns =[rheast_province.columns])
for i in post_harvest_years:
    df=rheast_province.loc[rheast_province['year']==i].reset_index(drop=True)
    rheast_province2=pd.concat([df])

# for i in post_harvest_years:
#     rheast_province2=pd.concat(rheast_province.loc[(rheast_province['year']==i)], axis=1)
    
# rheast_list_provinces=rheast_province['province'].to_list()
# rheast_set_provinces=set(rheast_list_provinces)
# rheast_provinces=list(rheast_set_provinces)

In [129]:
# Postharvest and rheas metrics


province_metrics2=[]
for i in provinces:
    post_province1=post_province.loc[post_province['province']==i].reset_index(drop=True)
    rheast_province2=rheast_province2.loc[rheast_province['province']==i].reset_index(drop=True)

    #  pearson's correlation
    corr=rheast_province2['yield'].corr(post_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheast_province2['yield']-post_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheast_province2['yield']-rheast_province2['yield'].mean())
                       -(post_province1['yield']-post_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheast_province2['yield']-post_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics2.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])

province_statistics2=pd.DataFrame(province_metrics2)
province_statistics2=province_statistics2.rename({0:'province',1: 
                                                  'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
# province_statistics.to_csv('preharvest_dssat_province_level_metrics.csv')
province_statistics2#.to_csv('zambia_post_harvest_province_level_metrics.csv')

province_statistics2

""


In [130]:
# pre_province1
# rheast_province
# post_province1
